In [2]:
import pandas as pd
import numpy as np
import faiss
from tqdm import tqdm_notebook

In [3]:
df_01 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_01_DEMO.csv')
df_02 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_02_PDDE.csv')
df_03 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_03_COP_U.csv')
df_04 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_04_PD_CLAC.csv')
df_05 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_05_BR.csv')
df_06 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_06_LPAY.csv')

/tmp/ipykernel_144438/2359529251.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_02 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_02_PDDE.csv')


In [3]:
df_01

,cust,ma_fem_dv,ages,zon_hlv
0,M000034966,여성,40대,Z07
1,M000059535,여성,30대,Z12
2,M000136117,여성,30대,Z11
3,M000201112,여성,50대,Z17
4,M000225114,여성,40대,Z17
...,...,...,...,...
29908,M999708287,여성,40대,Z10
29909,M999770689,여성,30대,Z16
29910,M999849895,여성,20대,Z04
29911,M999926092,남성,30대,Z08


In [4]:
df_05.drop(['cop_c', 'zon_hlv'], axis=1, inplace=True)
df_05.head(3)

,br_c,zon_mcls
0,A010001,Z17024
1,A010002,Z17018
2,A010003,Z17011


In [5]:
df_weather_cop = pd.read_csv('../dataset/df_cop_weather.csv')
df_weather_cop.drop(['Unnamed: 0'], axis=1, inplace=True)
df_weather_cop

,cust,cop_c,br_c,chnl_dv,de_hr,de_dt,buy_am,ratings,aver_temp
0,M430112881,A01,A010039,1,10,20210101,2,4.62,-3.4
1,M430112881,A01,A010039,1,10,20210101,3,4.62,-3.4
2,M430112881,A01,A010039,1,10,20210101,3,4.62,-3.4
3,M430112881,A01,A010039,1,10,20210101,3,4.62,-3.4
4,M506355556,A01,A010032,1,15,20210101,3,9.45,-3.4
...,...,...,...,...,...,...,...,...,...
4616641,M603445400,C01,C010087,2,19,20211118,3,9.20,9.8
4616642,M914963204,D01,D011330,1,14,20211118,1,7.20,9.8
4616643,M914963204,D01,D011308,1,16,20211118,1,6.30,9.8
4616644,M343659173,C01,C010043,1,21,20211118,1,9.90,9.8


In [6]:
df_weather_cop = pd.merge(df_weather_cop, df_01, on='cust')
df_weather_cop.drop(['de_dt', 'br_c'], axis=1, inplace=True)
df_weather_cop

,cust,cop_c,chnl_dv,de_hr,buy_am,ratings,aver_temp,ma_fem_dv,ages,zon_hlv
0,M430112881,A01,1,10,2,4.62,-3.4,여성,50대,Z17
1,M430112881,A01,1,10,3,4.62,-3.4,여성,50대,Z17
2,M430112881,A01,1,10,3,4.62,-3.4,여성,50대,Z17
3,M430112881,A01,1,10,3,4.62,-3.4,여성,50대,Z17
4,M430112881,A01,1,10,2,1.54,-3.4,여성,50대,Z17
...,...,...,...,...,...,...,...,...,...,...
4616641,M433359773,D01,1,18,1,9.20,4.5,여성,40대,Z10
4616642,M720176819,C01,1,13,2,9.40,4.5,여성,30대,Z17
4616643,M426777015,C01,2,13,3,9.20,4.5,남성,40대,Z10
4616644,M616600257,C01,2,10,1,9.20,9.8,남성,40대,Z16


In [7]:
df_weather_cop_x = df_weather_cop.drop('cust', axis=1)
df_weather_cop_y = df_weather_cop['cust']

In [8]:
df_weather_cop_x = pd.get_dummies(df_weather_cop_x)


In [9]:
df_weather_cop_x

,chnl_dv,de_hr,buy_am,ratings,aver_temp,cop_c_A01,cop_c_A02,cop_c_A03,cop_c_A04,cop_c_A05,...,zon_hlv_Z08,zon_hlv_Z09,zon_hlv_Z10,zon_hlv_Z11,zon_hlv_Z12,zon_hlv_Z13,zon_hlv_Z14,zon_hlv_Z15,zon_hlv_Z16,zon_hlv_Z17
0,1,10,2,4.62,-3.4,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,10,3,4.62,-3.4,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,10,3,4.62,-3.4,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,10,3,4.62,-3.4,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,10,2,1.54,-3.4,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4616641,1,18,1,9.20,4.5,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4616642,1,13,2,9.40,4.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4616643,2,13,3,9.20,4.5,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4616644,2,10,1,9.20,9.8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


### 차원 축소(PCA)

In [10]:
from sklearn.decomposition import PCA

def get_pca_data(data,n_components):
    pca = PCA(n_components=n_components)
    pca.fit(data)

    return pca.transform(data), pca
    
def get_pca_df(pca_data,pca):
    cols = ['pca_'+str(1+i) for i in range(pca.components_.shape[0])] 
    
    return pd.DataFrame(pca_data,columns=cols)

In [11]:
import numpy as np
import matplotlib.pyplot as plt

skca_pca10, pca10 = get_pca_data(df_weather_cop_x, 10)
print(np.sum(pca10.explained_variance_ratio_))

0.9922161777153882


In [14]:
df_weather_cop_x = pd.DataFrame(skca_pca10)
df_weather_cop_x_y = pd.concat([df_weather_cop_y, df_weather_cop_x], axis=1)
df_weather_cop_x_y

,cust,0,1,2,3,4,5,6,7,8,9
0,M430112881,16.739574,3.868843,3.996195,1.186170,-0.299832,-0.544229,0.638171,-0.526101,0.768618,-0.020255
1,M430112881,16.739938,3.866123,4.030989,2.162300,-0.284025,-0.534265,0.565096,-0.540374,0.777456,-0.073717
2,M430112881,16.739938,3.866123,4.030989,2.162300,-0.284025,-0.534265,0.565096,-0.540374,0.777456,-0.073717
3,M430112881,16.739938,3.866123,4.030989,2.162300,-0.284025,-0.534265,0.565096,-0.540374,0.777456,-0.073717
4,M430112881,16.765682,1.910107,6.370927,1.102809,-0.286181,-0.541351,0.678512,-0.515133,0.767205,-0.012736
...,...,...,...,...,...,...,...,...,...,...,...
4616641,M433359773,8.725324,0.630380,-4.557835,0.338446,-0.442311,0.763163,0.093883,0.630797,-0.058997,-0.319215
4616642,M720176819,8.771157,4.610296,-1.496841,1.226361,-0.267516,-0.700372,0.428490,-0.521524,-0.624131,-0.347958
4616643,M426777015,8.776445,4.489271,-1.307321,2.151935,0.984977,0.864674,-0.121330,0.618785,0.004454,-0.526671
4616644,M616600257,3.504738,6.817557,0.596880,0.147686,0.960913,0.768051,0.046507,-0.041797,-0.072319,0.523665


In [15]:
def get_vector(idx): 
    return pd.Series(df_weather_cop_x_y.iloc[idx].to_numpy())[0], pd.Series((df_weather_cop_x_y.iloc[idx].to_numpy())[1:]) 

In [16]:
from tqdm import tqdm

vector = []
for i in tqdm(range(len(df_weather_cop_x_y))):
    _, vec = get_vector(i)
    vector.append(np.array(vec))
    
vector = np.array(vector)
vector = vector.astype('float32')

100%|███████████████████████████████| 4616646/4616646 [16:13<00:00, 4742.36it/s]


### 유사도

In [18]:
res = faiss.StandardGpuResources()
index_flat = faiss.IndexFlatL2(vector.shape[1])
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)
gpu_index_flat.add(vector)

In [20]:
distances, indices = gpu_index_flat.search(vector, 6)

In [22]:
from collections import defaultdict
result = defaultdict(int)

for num in tqdm(range(len(df_weather_cop_x_y))):
    pname = df_weather_cop_x_y.iloc[num,:]['cust']
    similar = []
    for idx, loc in enumerate(indices[num][1:]):
        similar.append((df_weather_cop_x_y.iloc[loc,:]['cust'], distances[num][idx+1]))
    result[pname] = similar

100%|███████████████████████████████| 4616646/4616646 [43:14<00:00, 1779.47it/s]


In [33]:
result

defaultdict(int,
            {'M430112881': [('M265592092', 0.010879517),
              ('M101707311', 0.018081665),
              ('M501143162', 0.18217468),
              ('M826123316', 0.27035522),
              ('M538956000', 0.55755615)],
             'M506355556': [('M220433545', 0.13824463),
              ('M478937430', 0.22579956),
              ('M983523351', 0.24313354),
              ('M408281517', 0.29180908),
              ('M486895128', 0.33132935)],
             'M900002349': [('M889517267', 0.6565857),
              ('M623587035', 0.69314575),
              ('M712388981', 0.83070374),
              ('M544340024', 0.8765259),
              ('M223933059', 0.9086914)],
             'M694209751': [('M946939279', 0.040943146),
              ('M277564466', 0.054092407),
              ('M683321334', 0.09164429),
              ('M035569749', 0.11898041),
              ('M035569749', 0.12949371)],
             'M123475113': [('M325687895', 0.34791565),
              ('M167750458

In [31]:
len(result)

29756

In [32]:
similar

[('M200321071', 0.05025482),
 ('M701475461', 0.18590546),
 ('M586176066', 0.24071503),
 ('M407897828', 0.24656296),
 ('M794696154', 0.25004578)]

In [30]:
for idx, loc in enumerate(indices[5][1:]):
    print(idx, loc)

0 4140552
1 3366639
2 3186285
3 6
4 1594717


In [23]:
df_weather_cop_top5 = pd.DataFrame(columns=['cust','cust_prediction','cos'])
df_weather_cop_top5

,cust,cust_prediction,cos


In [24]:
for k, v in tqdm(result.items()):
    for p in v:
        tmp = [k]
        tmp.extend(list(p))
        df_len = len(df_weather_cop_top5)
        df_weather_cop_top5.loc[df_len] = tmp

df_weather_cop_top5

100%|█████████████████████████████████████| 29756/29756 [09:44<00:00, 50.87it/s]


,cust,cust_prediction,cos
0,M430112881,M265592092,0.010880
1,M430112881,M101707311,0.018082
2,M430112881,M501143162,0.182175
3,M430112881,M826123316,0.270355
4,M430112881,M538956000,0.557556
...,...,...,...
148775,M759754086,M200321071,0.050255
148776,M759754086,M701475461,0.185905
148777,M759754086,M586176066,0.240715
148778,M759754086,M407897828,0.246563


In [25]:
df_weather_cop_top5['cust']

0         M430112881
1         M430112881
2         M430112881
3         M430112881
4         M430112881
             ...    
148775    M759754086
148776    M759754086
148777    M759754086
148778    M759754086
148779    M759754086
Name: cust, Length: 148780, dtype: object

### LableEncoding

In [69]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

columns = ['cop_c', 'br_c', 'ma_fem_dv', 'ages', 'zon_hlv',]

for i in columns:
    globals()[f'encoder_{i}'] = LabelEncoder()
    df_weather_cop[i] = globals()[f'encoder_{i}'].fit_transform(df_weather_cop[i]) 

In [70]:
df_weather_cop.head(3)

,cust,cop_c,br_c,chnl_dv,de_hr,buy_am,ratings,aver_temp,ma_fem_dv,ages,zon_hlv
0,M430112881,0,38,1,10,2,4.62,-3.4,1,3,16
1,M430112881,0,38,1,10,3,4.62,-3.4,1,3,16
2,M430112881,0,38,1,10,3,4.62,-3.4,1,3,16


In [71]:
df_weather_cop_x = df_weather_cop.drop('cust', axis=1)
df_weather_cop_y = df_weather_cop['cust']

In [36]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df_weather_cop_x)
df_weather_cop_x_scaled = scaler.transform(df_weather_cop_x)

In [37]:
df_weather_cop_x = pd.DataFrame(df_weather_cop_x_scaled)
df_weather_cop_x.columns = ['cust', 'br_c', 'chnl_dv', 'de_hr', 'buy_am', 'ratings', 'aver_temp', 'ma_fem_dv', 'ages', 'zon_hlv']
df_weather_cop_x_y = pd.concat([df_weather_cop_y, df_weather_cop_x], axis=1)
df_weather_cop_x_y

,cop_c,cust,br_c,chnl_dv,de_hr,buy_am,ratings,aver_temp,ma_fem_dv,ages,zon_hlv
0,A01,-0.230580,-0.515200,-0.333472,-1.515806,1.006575,-0.198470,-1.767902,0.642411,0.932847,1.131824
1,A01,-0.230580,-0.515200,-0.333472,-1.515806,1.898933,-0.198470,-1.767902,0.642411,0.932847,1.131824
2,A01,-0.230580,-0.515200,-0.333472,-1.515806,1.898933,-0.198470,-1.767902,0.642411,0.932847,1.131824
3,A01,-0.230580,-0.515200,-0.333472,-1.515806,1.898933,-0.198470,-1.767902,0.642411,0.932847,1.131824
4,A01,-0.230580,-0.515200,-0.333472,-1.515806,1.006575,-1.026808,-1.767902,0.642411,0.932847,1.131824
...,...,...,...,...,...,...,...,...,...,...,...
4616641,D01,-0.219695,2.154923,-0.333472,0.626983,0.114218,1.033279,-0.930674,0.642411,-0.020444,-0.408763
4616642,C01,0.777365,1.772265,-0.333472,-0.712260,1.006575,1.087067,-0.930674,0.642411,-0.973734,1.131824
4616643,C01,-0.240192,1.810938,2.998750,-0.712260,1.898933,1.033279,-0.930674,-1.556637,-0.020444,-0.408763
4616644,C01,0.421388,1.800761,2.998750,-1.515806,0.114218,1.033279,-0.368990,-1.556637,-0.020444,0.911740


In [39]:
def get_vector(idx): 
    return pd.Series(df_weather_cop_x_y.iloc[idx].to_numpy())[0], pd.Series((df_weather_cop_x_y.iloc[idx].to_numpy())[1:]) 

In [40]:
from tqdm import tqdm

vector = []
for i in tqdm(range(len(df_weather_cop_x_y))):
    _, vec = get_vector(i)
    vector.append(np.array(vec))
    
vector = np.array(vector)
vector = vector.astype('float32')

100%|███████████████████████████████| 4616646/4616646 [16:30<00:00, 4659.66it/s]


In [41]:
vector.shape[1]

10

### 유사도 측정

In [42]:
res = faiss.StandardGpuResources()
index_flat = faiss.IndexFlatL2(vector.shape[1])
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)

In [43]:
gpu_index_flat.add(vector)

In [44]:
distances, indices = gpu_index_flat.search(vector, 6)

In [49]:
from collections import defaultdict
result = defaultdict(int)

for num in tqdm(range(len(df_weather_cop_x_y))):
    pname = df_weather_cop_x_y.iloc[num,:]['cop_c']
    similar = []
    for idx, loc in enumerate(indices[num][1:]):
        similar.append((df_weather_cop_x_y.iloc[loc,:]['cop_c'], distances[num][idx+1]))
    result[pname] = similar

100%|███████████████████████████████| 4616646/4616646 [43:43<00:00, 1759.68it/s]


In [59]:
pname = df_weather_cop_x_y.iloc[1,:]['cop_c']
pname

'A01'

In [60]:
for idx, loc in enumerate(indices[5][1:]):
        similar.append((df_weather_cop_x_y.iloc[loc,:]['cop_c'], distances[num][idx+1]))

In [61]:
similar

[('D01', 0.11894417),
 ('D01', 0.14680481),
 ('D01', 0.17437458),
 ('D01', 0.21159649),
 ('D01', 0.36488247),
 ('A03', 0.11894417),
 ('A02', 0.14680481),
 ('A02', 0.17437458),
 ('A02', 0.21159649),
 ('A02', 0.36488247)]

In [50]:
result

defaultdict(int,
            {'A01': [('A02', 0.06382942),
              ('A02', 0.0647831),
              ('A01', 0.0775156),
              ('A02', 0.091350555),
              ('A02', 0.10708618)],
             'A02': [('A03', 0.19748783),
              ('A02', 0.32273102),
              ('A02', 0.33424854),
              ('A02', 0.38493443),
              ('A02', 0.38493443)],
             'A03': [('A02', 0.033611298),
              ('A02', 0.03381443),
              ('A02', 0.06531048),
              ('A02', 0.06565857),
              ('A02', 0.07355595)],
             'D01': [('D01', 0.11894417),
              ('D01', 0.14680481),
              ('D01', 0.17437458),
              ('D01', 0.21159649),
              ('D01', 0.36488247)],
             'A04': [('A04', 0.014647007),
              ('A04', 0.040685654),
              ('A04', 0.0523448),
              ('A04', 0.052629948),
              ('A04', 0.053039074)],
             'D02': [('D02', 0.1708107),
              ('D01', 0.

In [55]:
df_weather_cop_top5 = pd.DataFrame(columns=['cop_c','cop_prediction','cos'])
df_weather_cop_top5

,cop_c,cop_prediction,cos


In [57]:
result.items()

dict_items([('A01', [('A02', 0.06382942), ('A02', 0.0647831), ('A01', 0.0775156), ('A02', 0.091350555), ('A02', 0.10708618)]), ('A02', [('A03', 0.19748783), ('A02', 0.32273102), ('A02', 0.33424854), ('A02', 0.38493443), ('A02', 0.38493443)]), ('A03', [('A02', 0.033611298), ('A02', 0.03381443), ('A02', 0.06531048), ('A02', 0.06565857), ('A02', 0.07355595)]), ('D01', [('D01', 0.11894417), ('D01', 0.14680481), ('D01', 0.17437458), ('D01', 0.21159649), ('D01', 0.36488247)]), ('A04', [('A04', 0.014647007), ('A04', 0.040685654), ('A04', 0.0523448), ('A04', 0.052629948), ('A04', 0.053039074)]), ('D02', [('D02', 0.1708107), ('D01', 0.2826357), ('D01', 0.2941742), ('D01', 0.34805107), ('D01', 0.36837482)]), ('C01', [('C01', 0.30059433), ('D01', 0.3041134), ('C01', 0.3153019), ('C01', 0.34650803), ('C01', 0.4310608)]), ('A05', [('C02', 0.10445404), ('C02', 0.17595673), ('D01', 0.24708939), ('D01', 0.34133148), ('D01', 0.4064827)]), ('A06', [('A02', 0.100839615), ('A03', 0.19098854), ('A03', 0.24

In [56]:
for k, v in tqdm(result.items()):
    for p in v:
        tmp = [k]
        tmp.extend(list(p))
        df_len = len(df_weather_cop_top5)
        df_weather_cop_top5.loc[df_len] = tmp

df_weather_cop_top5

100%|██████████████████████████████████████████| 12/12 [00:00<00:00, 193.05it/s]


,cop_c,cop_prediction,cos
0,A01,A02,0.063829
1,A01,A02,0.064783
2,A01,A01,0.077516
3,A01,A02,0.091351
4,A01,A02,0.107086
5,A02,A03,0.197488
6,A02,A02,0.322731
7,A02,A02,0.334249
8,A02,A02,0.384934
9,A02,A02,0.384934


In [8]:
x = df_weather_cop[['cust', 'chnl_dv', 'buy_am']]
y = df_weather_cop[['cop_c', 'de_hr', 'aver_temp']]

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler_x = MinMaxScaler()
x_scaled = scaler_x.fit_transform(x)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)

In [10]:
x_scaled.shape

(4616646, 3)

In [11]:
x_scaled = np.ascontiguousarray(x_scaled)
y_scaled = np.ascontiguousarray(y_scaled)
x_scaled = np.float32(x_scaled)
y_scaled = np.float32(y_scaled)

In [13]:
import numpy as np
import faiss

def create_and_search_index(embedding_size, user_embeddings, item_embeddings, k):
    index = faiss.IndexFlatL2(embedding_size)
    index.add(user_embeddings)

    D, I = index.search(item_embeddings, k=k) 

    return D, I
    
D, I = create_and_search_index(3, x_scaled[:200000], y_scaled[:200000], 5)

In [14]:
D

array([[0.20218286, 0.20218286, 0.20218286, 0.20218286, 0.20218286],
       [0.20218286, 0.20218286, 0.20218286, 0.20218286, 0.20218286],
       [0.20218286, 0.20218286, 0.20218286, 0.20218286, 0.20218286],
       ...,
       [0.02997303, 0.02997303, 0.02997303, 0.02997303, 0.02997327],
       [0.24169314, 0.24169314, 0.24169314, 0.24169314, 0.24169326],
       [0.24169314, 0.24169314, 0.24169314, 0.24169314, 0.24169326]],
      dtype=float32)

In [22]:
I[6865]

array([3062, 1320, 4509, 4690, 1734])